In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def CSPSA(psi_est, nu_ite, fun_teor, fun, s, t, a, A, b):
    """
    CSPSA
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of CSPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. CSPSA uses this function for the optimization.
        s, t, a, A, b: parametros CSPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by CSPSA.
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    valor_optimo = np.zeros((n_par, nu_ite))
    valor_optimo[:,0] = fun_teor(psi_est)

    for k in range(0,nu_ite-1):
        valor_optimo[:,k+1], psi_est = (itCSPSA(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, fun_teor ))

    return valor_optimo

In [3]:
def itCSPSA(z_k,  k, s, t, a, A, b, fun, fun_teor):
    """iteracion de CSPSA
    IN
        z_k: dim x n_rep. Cada columna es un estimador de dimension dim.
        k: iteracion
        s, t, a, A, b: parametros CSPSA
        fun: funcion a optimizar
    OUT
        z_k: dim x n_rep. Estimador actualizado
        z_value: real. valor de fun en el estimador z_k
    """
    dim = z_k.shape[0]
    rep = int(z_k.size/z_k.shape[0])
    alpha = a/(k+1+A)**s
    beta = b/(k+1)**t
    delta = (1j)**(np.random.randint(1,5,(dim, rep))).reshape(2,rep)
    z_k_mas = z_k + beta*delta
    z_k_menos = z_k - beta*delta
    fun_k_mas = fun(z_k_mas)
    fun_k_menos = fun(z_k_menos)
    grad = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta.conj())
    z_k = z_k - alpha*grad
    z_value = fun_teor(z_k)
    return z_value, z_k

In [4]:
def SPSA(psi_est, nu_ite, fun_teor, fun, s, t, a, A, b):
    """
    SPSA
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of SPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. SPSA uses this function for the optimization.
        s, t, a, A, b: parametros SPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by SPSA.
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    valor_optimo = np.zeros((n_par, nu_ite))
    valor_optimo[:,0] = fun_teor(psi_est)

    for k in range(0,nu_ite-1):
        valor_optimo[:,k+1], psi_est = (itSPSA(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, fun_teor ))


    return valor_optimo

In [5]:
def itSPSA(z_k,  k, s, t, a, A, b, fun, fun_teor):
    """iteracion de SPSA
    IN
        z_k: dim x n_rep. Cada columna es un estimador de dimension dim.
        k: iteracion
        s, t, a, A, b: parametros SPSA
        fun: funcion a optimizar
    OUT
        z_k: dim x n_rep. Estimador actualizado
        z_value: real. valor de fun en el estimador z_k
    """
    dim = z_k.shape[0]
    rep = int(z_k.size/z_k.shape[0])
    alpha = a/(k+1+A)**s
    beta = b/(k+1)**t
    
    #Cambiamos un delta complejo por uno real (+1,-1)
    
    delta = (-1)**(np.random.randint(1,5,(dim, rep))).reshape(2,rep)
    delta_i = (-1)**(np.random.randint(1,5,(dim, rep))).reshape(2,rep)
    
    #Definimos parte real e imaginaria
    
    z_k_mas_r = np.real(z_k) + beta*delta
    z_k_menos_r = np.real(z_k) - beta*delta
    
    z_k_mas_i = np.imag(z_k) + beta*delta_i
    z_k_menos_i = np.imag(z_k) - beta*delta_i
    

    #Nuevo estado, normalizado
    
    z_k_mas = z_k_mas_r +1j*z_k_mas_i
    z_k_menos = z_k_menos_r+1j*z_k_menos_i
    
    z_k_mas = z_k_mas/np.linalg.norm(z_k_mas, axis=0)
    z_k_menos = z_k_menos/np.linalg.norm(z_k_menos, axis=0)
    
    fun_k_mas = fun(z_k_mas)
    fun_k_menos = fun(z_k_menos)
    
    #Gradiente real e imaginaria
    
    grad = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta)
    grad_i = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta_i)
    
    z_k = z_k - alpha*grad - 1j*alpha*grad_i
    z_k = z_k/np.linalg.norm(z_k, axis=0)
    
    z_value = fun_teor(z_k)
    return z_value, z_k

In [6]:
def SPSA_ang(psi_est, nu_ite, fun_teor, fun, s, t, a, A, b):
    """
    SPSA
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of SPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. SPSA uses this function for the optimization.
        s, t, a, A, b: parametros SPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by SPSA.
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    valor_optimo = np.zeros((n_par, nu_ite))
    valor_optimo[:,0] = fun_teor(psi_est)

    for k in range(0,nu_ite-1):
        valor_optimo[:,k+1], psi_est = (itSPSA_ang(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, fun_teor ))


    return valor_optimo

In [7]:
def itSPSA_ang(z_k, k, s, t, a, A, b, fun, fun_teor):
    """iteracion de SPSA
    IN
        z_k: dim x n_rep. Cada columna es un estimador de dimension dim.
        k: iteracion
        s, t, a, A, b: parametros SPSA
        fun: funcion a optimizar
    OUT
        z_k: dim x n_rep. Estimador actualizado
        z_value: real. valor de fun en el estimador z_k
    """
    
    alpha = a/(k+1+A)**s
    beta = b/(k+1)**t
    
    # Definir deltas para cada variable
    
    delta_t = (-1)**(np.random.randint(1,5))
    delta_p = (-1)**(np.random.randint(1,5))
    
    #obtener theta y phase
    
    z1 = z_k[0]
    r1 = np.absolute(z1)
    phi1 = np.angle(z1)

    z2 = z_k[1]
    r2 = np.absolute(z2)
    phi2 = np.angle(z2)

    R = np.divide(r2,r1)

    phase = float(phi2-phi1)
    theta = float(np.arctan(R))

    #sumamos y restamos delta
    
    theta_mas = theta + beta*delta_t
    theta_menos = theta - beta*delta_t
    
    phase_mas = phase + beta*delta_p
    phase_menos = phase - beta*delta_p
    
    # formamos estados normalizados para cada par theta,phase
    
    a1_mas = np.cos(theta_mas)
    a2_mas = np.sin(theta_mas)*(np.cos(phase_mas)+1j*np.sin(phase_mas))
    
    a1_menos = np.cos(theta_menos)
    a2_menos = np.sin(theta_menos)*(np.cos(phase_menos)+1j*np.sin(phase_menos))

    psi_bloch_mas = np.array([[a1_mas],[a2_mas]])
    psi_bloch_menos = np.array([[a1_menos],[a2_menos]])
    
    psi_bloch_mas = psi_bloch_mas/np.linalg.norm(psi_bloch_mas, axis=0)
    psi_bloch_menos = psi_bloch_menos/np.linalg.norm(psi_bloch_menos, axis=0)
    
    fun_k_mas = fun(psi_bloch_mas)
    fun_k_menos = fun(psi_bloch_menos)
    
   
    #Gradiente para ambas variables
    
    grad_t = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta_t)
    grad_p = np.divide(fun_k_mas - fun_k_menos, 2*beta*delta_p)
    
    theta = theta - alpha*float(grad_t)
    phase = phase - alpha*float(grad_p)
    
    a1 = np.cos(theta)
    a2 = np.sin(theta)*(np.cos(phase)+1j*np.sin(phase))

    z_k = np.array([[a1],[a2]])
    z_k = z_k/np.linalg.norm(z_k, axis=0)
    
    z_value = fun_teor(z_k)
    return z_value, z_k

In [8]:
# Definir Estado

def estado(dim, n_par):
    psi = (np.random.normal(loc=0.0, scale=1.0,
           size=(dim, n_par))
           + np.random.normal(loc=0.0, scale=1.0,
           size=(dim, n_par))*1j)
    psi = psi/np.linalg.norm(psi, axis=0)
    return psi

In [9]:
# Estado en representacion de Bloch

def est_bloch(psi_est):
    z1 = psi_est[0]
    r1 = np.absolute(z1)
    phi1 = np.angle(z1)

    z2 = psi_est[1]
    r2 = np.absolute(z2)
    phi2 = np.angle(z2)

    R = np.divide(r2,r1)

    phase = phi2-phi1
    theta = np.arctan(R)

    a1 = np.cos(theta)
    a2 = np.sin(theta)*(np.cos(phase)+1j*np.sin(phase))

    psi_bloch = np.array([a1,a2])
    psi_bloch = psi_bloch/np.linalg.norm(psi_bloch, axis=0)
    return psi_bloch, phase, theta

In [10]:
# Definir Hamiltoniano

def H_paul (A,B,C):
    pauli_x = np.array((((0, 1), (1, 0))))
    pauli_y = np.array((((0, -1j), (1j, 0))))
    pauli_z = np.array((((1, 0), (0, -1))))
    H_paul = A*pauli_x + B*pauli_y + C*pauli_z
    return H_paul

In [11]:
# Definir Valor de expectacion

def exp_H (psi_est, H):
    psi = np.matrix(psi_est)
    psi = psi/np.linalg.norm(psi, axis=0)
    psi_dagger = psi.getH()
    H = np.matrix(H)
    exp_H = psi_dagger*H*psi
    
    # calculamos valor absoluto, para eliminar parte imaginaria producto de errores
    exp_H = np.real(exp_H)
    
    return exp_H

In [12]:
def ganancia_a_CSPSA(z_k, l, fun, t, b):
    
    
    dim = z_k.shape[0]
    rep = int(z_k.size/z_k.shape[0])
    beta = b/(1)**t
    diff = np.empty((l,1,1), dtype=np.single)
    
    for i in range(0,l):
    
        delta = (1j)**(np.random.randint(1,5,(dim, rep))).reshape(2,rep)
        z_k_mas = z_k + beta*delta
        z_k_menos = z_k - beta*delta
        
        fun_k_mas = fun(z_k_mas)
        fun_k_menos = fun(z_k_menos)
        
        diff[i] = abs(fun_k_mas-fun_k_menos)    
        
    prom = np.sum(diff, axis=0)/l
    
    a_arr = np.divide(2*np.pi*b, 5*prom)
    a_CSPSA = float(a_arr[0])
    
    return a_CSPSA

In [13]:
def ganancia_a_SPSA(z_k, l, fun, t, b):
    
    
    dim = z_k.shape[0]
    rep = int(z_k.size/z_k.shape[0])
    beta = b/(1)**t
    diff = np.empty((l,1,1), dtype=np.single)
    
    for i in range(0,l):
    
        delta = (-1)**(np.random.randint(1,5,(dim, rep))).reshape(2,rep)
        delta_i = (-1)**(np.random.randint(1,5,(dim, rep))).reshape(2,rep)
    
        #Definimos parte real e imaginaria
    
        z_k_mas_r = np.real(z_k) + beta*delta
        z_k_menos_r = np.real(z_k) - beta*delta
    
        z_k_mas_i = np.imag(z_k) + beta*delta_i
        z_k_menos_i = np.imag(z_k) - beta*delta_i
    

        #Nuevo estado, normalizado
    
        z_k_mas = z_k_mas_r +1j*z_k_mas_i
        z_k_menos = z_k_menos_r+1j*z_k_menos_i
    
        z_k_mas = z_k_mas/np.linalg.norm(z_k_mas, axis=0)
        z_k_menos = z_k_menos/np.linalg.norm(z_k_menos, axis=0)
    
        fun_k_mas = fun(z_k_mas)
        fun_k_menos = fun(z_k_menos)
        
        diff[i] = abs(fun_k_mas-fun_k_menos)    
        
    prom = np.sum(diff, axis=0)/l
    
    a_arr = np.divide(2*np.pi*b, 5*prom)
    a_SPSA = float(a_arr[0])
    
    return a_SPSA

In [14]:
def ganancia_a_SPSA_ang(z_k, l, fun, t, b):
    
    
    dim = z_k.shape[0]
    rep = int(z_k.size/z_k.shape[0])
    beta = b/(1)**t
    diff = np.empty((l,1,1), dtype=np.single)
    
    for i in range(0,l):
    
        # Definir deltas para cada variable
    
        delta_t = (-1)**(np.random.randint(1,5))
        delta_p = (-1)**(np.random.randint(1,5))
    
        #obtener theta y phase
    
        z1 = z_k[0]
        r1 = np.absolute(z1)
        phi1 = np.angle(z1)

        z2 = z_k[1]
        r2 = np.absolute(z2)
        phi2 = np.angle(z2)

        R = np.divide(r2,r1)

        phase = float(phi2-phi1)
        theta = float(np.arctan(R))

        #sumamos y restamos delta
    
        theta_mas = theta + beta*delta_t
        theta_menos = theta - beta*delta_t
    
        phase_mas = phase + beta*delta_p
        phase_menos = phase - beta*delta_p
    
        # formamos estados normalizados para cada par theta,phase
    
        a1_mas = np.cos(theta_mas)
        a2_mas = np.sin(theta_mas)*(np.cos(phase_mas)+1j*np.sin(phase_mas))
    
        a1_menos = np.cos(theta_menos)
        a2_menos = np.sin(theta_menos)*(np.cos(phase_menos)+1j*np.sin(phase_menos))

        psi_bloch_mas = np.array([[a1_mas],[a2_mas]])
        psi_bloch_menos = np.array([[a1_menos],[a2_menos]])
    
        psi_bloch_mas = psi_bloch_mas/np.linalg.norm(psi_bloch_mas, axis=0)
        psi_bloch_menos = psi_bloch_menos/np.linalg.norm(psi_bloch_menos, axis=0)
    
        fun_k_mas = fun(psi_bloch_mas)
        fun_k_menos = fun(psi_bloch_menos)
        
        diff[i] = abs(fun_k_mas-fun_k_menos)    
        
    prom = np.sum(diff, axis=0)/l
    
    a_arr = np.divide(2*np.pi*b, 5*prom)
    a_SPSA_ang = float(a_arr[0])
    
    return a_SPSA_ang

In [15]:
def CSPSA2(psi_est, nu_ite, fun_teor, fun, s, t, A, b):
    """
    CSPSA2
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of CSPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. CSPSA uses this function for the optimization.
        s, t, a, A, b: parametros CSPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by CSPSA.
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    valor_optimo = np.zeros((n_par, nu_ite))
    valor_optimo[:,0] = fun_teor(psi_est)
    
    a = ganancia_a_CSPSA(psi_est, 25, fun, t, b)

    for k in range(0,nu_ite-1):
        valor_optimo[:,k+1], psi_est = (itCSPSA(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, fun_teor ))

    return valor_optimo

In [16]:
def SPSA2(psi_est, nu_ite, fun_teor, fun, s, t, A, b):
    """
    SPSA
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of SPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. SPSA uses this function for the optimization.
        s, t, a, A, b: parametros SPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by SPSA.
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    valor_optimo = np.zeros((n_par, nu_ite))
    valor_optimo[:,0] = fun_teor(psi_est)
    
    a = ganancia_a_SPSA(psi_est, 25, fun, t, b)

    for k in range(0,nu_ite-1):
        valor_optimo[:,k+1], psi_est = (itSPSA(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, fun_teor ))


    return valor_optimo

In [17]:
def SPSA_ang2(psi_est, nu_ite, fun_teor, fun, s, t, A, b):
    """
    SPSA
    IN
        psi_est: dim x n_par matrix. dim is de dimension of each vector, and n_par
                the number of guesses in parallel.
        nu_ite: int. Number of iterations of SPSA.
        fun_teor: function. It is the known function, for example, the theoretical
                 fidelity between the unknown state of the system and the guess.
        fun: function. SPSA uses this function for the optimization.
        s, t, a, A, b: parametros SPSA
    OUT
        valor_optimo: real. Optimal value of fun finded by SPSA.
    """

    n_par = int(psi_est.size/psi_est.shape[0])
    valor_optimo = np.zeros((n_par, nu_ite))
    valor_optimo[:,0] = fun_teor(psi_est)
    
    a = ganancia_a_SPSA_ang(psi_est, 25, fun, t, b)

    for k in range(0,nu_ite-1):
        valor_optimo[:,k+1], psi_est = (itSPSA_ang(psi_est/np.linalg.norm(psi_est, axis = 0),
                                       k, s, t, a, A, b, fun, fun_teor ))


    return valor_optimo